# Attention is all you need. Трансформеры.

## Библиотеки

In [ ]:
from copy import deepcopy

import matplotlib.pyplot as plt
from matplotlib.image import imread
from mpl_toolkits import mplot3d
from matplotlib import gridspec
from PIL import Image
import io
import os
from urllib.request import urlopen
from skimage.segmentation import mark_boundaries
from nltk.tokenize import RegexpTokenizer

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import requests
from scipy.stats import norm
import torch

from sklearn.metrics import classification_report
from torch.utils.tensorboard import SummaryWriter


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

## Код для обучения

## Модель внимания в рекурентных моделях

## Трансформер 

## Переводчик Tensor2Tensor

## Токенизация BPE (Byte Pair Encoding)

## BERT (Bidirectional Encoder Representations from Transformers)

## Модель LaBSE (Language-agnostic BERT Sentence Embedding)